## Кластеризация набора данных

In [1]:
import pandas as pd
import numpy as np

In [2]:
df=pd.read_csv("result_dataset.csv", sep="\t", encoding="utf-16")
df['значимость_раздела_wsss'] = pd.to_numeric(df['значимость_раздела_wsss'],errors='coerce')
df["значимость_раздела_wsss"]=df["значимость_раздела_wsss"].fillna(float(int(df["значимость_раздела_wsss"].mean())))
df.drop(df.iloc[:,8:],inplace=True,axis=1)
df.head()

,название_компетенции_wsss,описание_компетенции_wsss,название_раздела_wsss,значимость_раздела_wsss,признаки_раздела_wsss,count_description,count_title,count_titlefeat
0,Ювелирное дело,профессия ювелира прежде связана ручным трудом...,обработка поверхности,15.0,специалист должен знать понимать специализиров...,256,2,48
1,Ювелирное дело,профессия ювелира прежде связана ручным трудом...,изготовление сложных элементов ювелирных издел...,20.0,специалист должен знать понимать способы испол...,256,9,38
2,Ювелирное дело,профессия ювелира прежде связана ручным трудом...,изготовление простых элементов ювелирных изделий,20.0,специалист должен знать понимать способы испол...,256,5,15
3,Ювелирное дело,профессия ювелира прежде связана ручным трудом...,подготовка драгоценных сплавов изготовления эл...,10.0,специалист должен знать понимать свойства спос...,256,7,21
4,Ювелирное дело,профессия ювелира прежде связана ручным трудом...,изготовление сплавов драгоценных металлов,5.0,специалист должен знать понимать состав сплаво...,256,4,56


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 806 entries, 0 to 805
Data columns (total 8 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   название_компетенции_wsss  806 non-null    object 
 1   описание_компетенции_wsss  806 non-null    object 
 2   название_раздела_wsss      806 non-null    object 
 3   значимость_раздела_wsss    806 non-null    float64
 4   признаки_раздела_wsss      806 non-null    object 
 5   count_description          806 non-null    int64  
 6   count_title                806 non-null    int64  
 7   count_titlefeat            806 non-null    int64  
dtypes: float64(1), int64(3), object(4)
memory usage: 50.5+ KB


In [4]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer

class TextSelector(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
    def fit(self, X):
        return self
    def transform(self, X, y=None):
        return X[self.key]
    
class NumericalSelector(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
    def fit(self, X):
        return self
    def transform(self, X, y=None):
        return X[[self.key]]

In [5]:
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("russian") 

def stemming_tokenizer(text):
    text = re.split('\W+', text)
    text = [stemmer.stem(word) for word in text]
    return text

Для того, чтобы можно было использовать все признаки, воспользуемся `Pipeline`, `BaseEstimator, TransformerMixin`.

In [6]:
from nltk.corpus import stopwords
import re
stop_words = stopwords.words("russian")

dictionary = Pipeline([
    ('selector', TextSelector(key="описание_компетенции_wsss")),
    ('tfidf', TfidfVectorizer(stop_words = stop_words,tokenizer=stemming_tokenizer))
    ])

dictionary.fit_transform(df)

/opt/anaconda3/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:383: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['бол', 'больш', 'будт', 'быт', 'вед', 'впроч', 'всег', 'всегд', 'даж', 'друг', 'е', 'ег', 'ем', 'есл', 'ест', 'ещ', 'зач', 'зде', 'ил', 'иногд', 'когд', 'конечн', 'куд', 'лучш', 'межд', 'мен', 'мног', 'мо', 'можн', 'нег', 'нельз', 'нибуд', 'никогд', 'нич', 'опя', 'посл', 'пот', 'почт', 'разв', 'сво', 'себ', 'совс', 'теб', 'тепер', 'тог', 'тогд', 'тож', 'тольк', 'хорош', 'хот', 'чег', 'чут', 'эт'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


<806x3417 sparse matrix of type '<class 'numpy.float64'>'
	with 119453 stored elements in Compressed Sparse Row format>

In [7]:
title = Pipeline([
    ('selector', TextSelector(key="название_раздела_wsss")),
    ('tfidf', TfidfVectorizer(stop_words = stop_words,tokenizer=stemming_tokenizer))
    ])

title.fit_transform(df)

<806x811 sparse matrix of type '<class 'numpy.float64'>'
	with 3512 stored elements in Compressed Sparse Row format>

In [8]:
ves_title = Pipeline([
    ('selector', NumericalSelector(key="значимость_раздела_wsss")),
    ('tfidf', StandardScaler())
    ])

ves_title.fit_transform(df)

array([[-0.05145834],
       [ 0.40835783],
       [ 0.40835783],
       [-0.5112745 ],
       [-0.97109067],
       [-0.5112745 ],
       [ 0.40835783],
       [ 0.868174  ],
       [-0.05145834],
       [ 0.40835783],
       [-0.05145834],
       [-0.32734803],
       [-0.2353848 ],
       [ 0.868174  ],
       [-0.05145834],
       [ 0.40835783],
       [-0.05145834],
       [-0.32734803],
       [-0.2353848 ],
       [-0.05145834],
       [ 0.868174  ],
       [ 1.32799016],
       [-0.97109067],
       [-0.5112745 ],
       [-0.5112745 ],
       [-0.97109067],
       [-0.05145834],
       [ 0.868174  ],
       [ 1.32799016],
       [-0.97109067],
       [-0.5112745 ],
       [-0.5112745 ],
       [-0.97109067],
       [-0.32734803],
       [ 0.13246813],
       [ 1.1440637 ],
       [-0.5112745 ],
       [-0.69520097],
       [ 0.868174  ],
       [ 0.77621076],
       [ 0.89576297],
       [ 0.5922843 ],
       [ 0.56469533],
       [-0.7871642 ],
       [ 0.868174  ],
       [ 0

In [9]:
titlefeat = Pipeline([
    ('selector', TextSelector(key="признаки_раздела_wsss")),
    ('tfidf', TfidfVectorizer(stop_words = stop_words,tokenizer=stemming_tokenizer))
    ])

titlefeat.fit_transform(df)

/opt/anaconda3/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:383: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['бол', 'больш', 'будт', 'быт', 'вед', 'впроч', 'всег', 'всегд', 'даж', 'друг', 'е', 'ег', 'ем', 'есл', 'ест', 'ещ', 'зач', 'зде', 'ил', 'иногд', 'когд', 'конечн', 'куд', 'лучш', 'межд', 'мен', 'мног', 'мо', 'можн', 'нег', 'нельз', 'нибуд', 'никогд', 'нич', 'опя', 'посл', 'пот', 'почт', 'разв', 'сво', 'себ', 'совс', 'теб', 'тепер', 'тог', 'тогд', 'тож', 'тольк', 'хорош', 'хот', 'чег', 'чут', 'эт'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


<806x4808 sparse matrix of type '<class 'numpy.float64'>'
	with 44673 stored elements in Compressed Sparse Row format>

In [10]:
from sklearn.pipeline import FeatureUnion

feat_full = FeatureUnion([("dictionary", dictionary),
                    ("titlefeat", titlefeat)])
feat_processing_full=Pipeline([('feat_full', feat_full)])
X_full=feat_full.fit_transform(df).toarray()

### Использование метрик
В данной ситуации при обучении без учителя лучше использовать метрику `silhouette_score`

In [11]:
from sklearn.metrics import silhouette_score

### PCA
Метод главных компонент (Principal Component Analysis) — один из самых интуитивно простых и часто используемых методов для снижения размерности данных и проекции их на ортогональное подпространство признаков

In [12]:
from sklearn.decomposition import PCA

pca=PCA(n_components=2, random_state=0)
pca_fit=pca.fit_transform(X_full)

На основе результата `silhouette_score` будет выбрана модель кластеризации. Чем выше, тем лучше

### KNN

In [13]:
from sklearn.cluster import KMeans
model_kmeans = KMeans(n_clusters=7,random_state=0)
model_kmeans.fit(X_full)
pred_kmeans=model_kmeans.predict(X_full)
silhouette_score(pca_fit, pred_kmeans)

-0.14293034529868145

## Агломеративная кластеризация

In [14]:
from sklearn.cluster import AgglomerativeClustering
model_agg = AgglomerativeClustering(n_clusters=7)
model_agg.fit(X_full)
pred_model_agg=model_agg.labels_
silhouette_score(pca_fit, pred_model_agg)

-0.04782393777554954

## Спектральная кластеризация

In [15]:
from sklearn.cluster import SpectralClustering
model_sc = SpectralClustering(n_clusters=7)
model_sc.fit(X_full)
pred_model_sc=model_agg.labels_
silhouette_score(pca_fit, pred_model_sc)

-0.04782393777554954

Делая вывод на основе этих значений, можно сказать, что лучше всего себя показал метод KMeans с производительностью 0.14

In [16]:
df["cluster_kmeans"]=pred_kmeans

## Визуализация зависимостей n-грамм в данных

In [17]:
df.head()

,название_компетенции_wsss,описание_компетенции_wsss,название_раздела_wsss,значимость_раздела_wsss,признаки_раздела_wsss,count_description,count_title,count_titlefeat,cluster_kmeans
0,Ювелирное дело,профессия ювелира прежде связана ручным трудом...,обработка поверхности,15.0,специалист должен знать понимать специализиров...,256,2,48,3
1,Ювелирное дело,профессия ювелира прежде связана ручным трудом...,изготовление сложных элементов ювелирных издел...,20.0,специалист должен знать понимать способы испол...,256,9,38,3
2,Ювелирное дело,профессия ювелира прежде связана ручным трудом...,изготовление простых элементов ювелирных изделий,20.0,специалист должен знать понимать способы испол...,256,5,15,3
3,Ювелирное дело,профессия ювелира прежде связана ручным трудом...,подготовка драгоценных сплавов изготовления эл...,10.0,специалист должен знать понимать свойства спос...,256,7,21,3
4,Ювелирное дело,профессия ювелира прежде связана ручным трудом...,изготовление сплавов драгоценных металлов,5.0,специалист должен знать понимать состав сплаво...,256,4,56,3


In [18]:
feat = FeatureUnion([("dictionary", dictionary),
                        ("titlefeat", titlefeat),
                         ("title", title),
                         ("ves_title", ves_title)])
feat_processing=Pipeline([('feat', feat)])
X=feat.fit_transform(df).toarray()

In [19]:
from sklearn.cluster import KMeans
model_kmeans = KMeans(n_clusters=7,random_state=0)
model_kmeans.fit(X)
pred_kmeans=model_kmeans.predict(X)
df["cluster_kmeans_full"]=pred_kmeans
df.head()

,название_компетенции_wsss,описание_компетенции_wsss,название_раздела_wsss,значимость_раздела_wsss,признаки_раздела_wsss,count_description,count_title,count_titlefeat,cluster_kmeans,cluster_kmeans_full
0,Ювелирное дело,профессия ювелира прежде связана ручным трудом...,обработка поверхности,15.0,специалист должен знать понимать специализиров...,256,2,48,3,5
1,Ювелирное дело,профессия ювелира прежде связана ручным трудом...,изготовление сложных элементов ювелирных издел...,20.0,специалист должен знать понимать способы испол...,256,9,38,3,2
2,Ювелирное дело,профессия ювелира прежде связана ручным трудом...,изготовление простых элементов ювелирных изделий,20.0,специалист должен знать понимать способы испол...,256,5,15,3,2
3,Ювелирное дело,профессия ювелира прежде связана ручным трудом...,подготовка драгоценных сплавов изготовления эл...,10.0,специалист должен знать понимать свойства спос...,256,7,21,3,3
4,Ювелирное дело,профессия ювелира прежде связана ручным трудом...,изготовление сплавов драгоценных металлов,5.0,специалист должен знать понимать состав сплаво...,256,4,56,3,3


Таким образом видно, что используя все атрибуты получается так, что одна компетенция может входить в разные блоки. Визуализируем кол-во кластерных пересечений `cluster_full` и `cluster_kmeans_full`

In [20]:
dictionary_nrgam2 = Pipeline([
    ('selector', TextSelector(key="описание_компетенции_wsss")),
    ('tfidf', TfidfVectorizer(stop_words = stop_words,tokenizer=stemming_tokenizer, ngram_range=(1,2)))
    ])

dictionary_nrgam2.fit_transform(df)

/opt/anaconda3/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:383: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['бол', 'больш', 'будт', 'быт', 'вед', 'впроч', 'всег', 'всегд', 'даж', 'друг', 'е', 'ег', 'ем', 'есл', 'ест', 'ещ', 'зач', 'зде', 'ил', 'иногд', 'когд', 'конечн', 'куд', 'лучш', 'межд', 'мен', 'мног', 'мо', 'можн', 'нег', 'нельз', 'нибуд', 'никогд', 'нич', 'опя', 'посл', 'пот', 'почт', 'разв', 'сво', 'себ', 'совс', 'теб', 'тепер', 'тог', 'тогд', 'тож', 'тольк', 'хорош', 'хот', 'чег', 'чут', 'эт'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


<806x17728 sparse matrix of type '<class 'numpy.float64'>'
	with 283196 stored elements in Compressed Sparse Row format>

In [21]:
titlefeat_ngram2 = Pipeline([
    ('selector', TextSelector(key="признаки_раздела_wsss")),
    ('tfidf', TfidfVectorizer(stop_words = stop_words,tokenizer=stemming_tokenizer, ngram_range=(1,2)))
    ])

titlefeat_ngram2.fit_transform(df)

<806x31028 sparse matrix of type '<class 'numpy.float64'>'
	with 100866 stored elements in Compressed Sparse Row format>

In [22]:
from sklearn.pipeline import FeatureUnion
feat_full_ngram2 = FeatureUnion([("dictionary_nrgam2", dictionary_nrgam2),
                    ("titlefeat_ngram2", titlefeat_ngram2)])

feat_processing_full_ngram2=Pipeline([('feat_full_ngram2', feat_full_ngram2)])
X_full_ngram2=feat_full_ngram2.fit_transform(df).toarray()

In [23]:
from sklearn.cluster import KMeans
model_kmeans_ngram2 = KMeans(n_clusters=7,random_state=0)
model_kmeans_ngram2.fit(X_full_ngram2)
pred_model_kmeans_ngram2=model_kmeans_ngram2.predict(X_full_ngram2)
df["cluster_ngram2"]=pred_model_kmeans_ngram2

In [24]:
lst1=df["cluster_kmeans"].value_counts(sort=False).tolist()
lst2=df["cluster_ngram2"].value_counts(sort=False).tolist()

### Наименование кластеров

In [25]:
sett=df[df['cluster_kmeans'] == 0]
sett.drop(sett.iloc[:,1:],inplace=True,axis=1)
comp=pd.unique(sett["название_компетенции_wsss"]).tolist()
comp

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


['Электромонтаж (Юниоры)',
 'Электромонтаж',
 'Туризм (Юниоры)',
 'Туризм',
 'Турагентская деятельность',
 'Сервис на воздушном транспорте (Юниоры)',
 'Сервис на воздушном транспорте',
 'Ресторанный сервис (Юниоры)',
 'Ресторанный сервис',
 'Поварское дело (Юниоры)',
 'Поварское дело',
 'Администрирование отеля(Юниоры)',
 'Администрирование отеля']

In [26]:
sett=df[df['cluster_kmeans'] == 1]
sett.drop(sett.iloc[:,1:],inplace=True,axis=1)
comp=pd.unique(sett["название_компетенции_wsss"]).tolist()
comp

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


['Управление фронтальным погрузчиком',
 'Управление бульдозером',
 'Фотография (Юниоры)',
 'Фотография',
 'Управление автогрейдером',
 'Сухое строительство и штукатурные работы (Юниоры)',
 'Сухое строительство и штукатурные работы',
 'Ремонт и обслуживание легковых автомобилей (Юниоры)',
 'Ремонт и обслуживание легковых автомобилей',
 'Печатные технологии в прессе (Юниоры)',
 'Печатные технологии в прессе',
 'Обслуживание тяжелой техники',
 'Лазерные технологии (Lasertechnology)',
 'Лабораторный химический анализ',
 'Звукорежиссура (Юниоры)',
 'Звукорежиссура',
 'Графический дизайн (Юниоры)',
 'Графический дизайн',
 'Ветеринария (Юниоры)',
 'Ветеринария',
 'Промышленная механика и монтаж',
 'Промышленная робототехника',
 'Сварочные технологии',
 'Сельскохозяйственные биотехнологии',
 'Синтез и обработка минералов',
 'Сити-фермерство',
 'Токарные работы на станках с ЧПУ',
 'Фрезерные работы на станках с ЧПУ',
 'Водные технологии',
 'Геодезия(юниоры)',
 'Геодезия(16-22)']

In [27]:
sett=df[df['cluster_kmeans'] == 2]
sett.drop(sett.iloc[:,1:],inplace=True,axis=1)
comp=pd.unique(sett["название_компетенции_wsss"]).tolist()
comp

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


['Ветеринария (Юниоры)',
 'Ветеринария',
 'Физическая культура, спорт и фитнес(юниоры)',
 'Физическая культура, спорт и фитнес',
 'Преподавание в младших классах(юниоры)',
 'Дошкольное воспитание',
 'Дошкольное воспитание(юниоры)',
 'Преподавание в младших классах']

In [28]:
sett=df[df['cluster_kmeans'] == 3]
sett.drop(sett.iloc[:,1:],inplace=True,axis=1)
comp=pd.unique(sett["название_компетенции_wsss"]).tolist()
comp

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


['Ювелирное дело',
 'Флористика (Юниоры)',
 'Флористика',
 'Технологии моды (Юниоры)',
 'Технологии моды',
 'Реставрация произведений из дерева',
 'Производство мебели (Юниоры)',
 'Производство мебели',
 'Промышленный дизайн (Юниоры)',
 'Промышленный дизайн',
 'Кондитерское дело (Юниоры)',
 'Кондитерское дело',
 'Дизайн интерьера (Юниоры)',
 'Дизайн интерьера',
 'Графический дизайн (Юниоры)',
 'Графический дизайн',
 'Визуальный мерчендайзинг (Юниоры)',
 'Визуальный мерчендайзинг',
 '3D моделирование для компьютерных игр',
 'Технологии композитов',
 'Командная работа на производстве ',
 'Изготовление изделий из полимерных материалов',
 'Кровельные работы по металлу',
 'Кровельные работы',
 'Архитектурная обработка камня']

In [29]:
sett=df[df['cluster_kmeans'] == 4]
sett.drop(sett.iloc[:,1:],inplace=True,axis=1)
comp=pd.unique(sett["название_компетенции_wsss"]).tolist()
comp

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


['Хлебопечение (Юниоры)',
 'Хлебопечение',
 'Выпечка осетинских пирогов (Юниоры)',
 'Выпечка осетинских пирогов']

In [30]:
sett=df[df['cluster_kmeans'] == 5]
sett.drop(sett.iloc[:,1:],inplace=True,axis=1)
comp=pd.unique(sett["название_компетенции_wsss"]).tolist()
comp

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


['Эстетическая косметология (Юниоры)',
 'Эстетическая косметология',
 'Электромонтаж (Юниоры)',
 'Электромонтаж',
 'Сантехника и отопление (Юниоры)',
 'Сантехника и отопление',
 'Парикмахерское искусство (Юниоры)',
 'Парикмахерское искусство',
 'Медицинский и социальный уход (Юниоры)',
 'Медицинский и социальный уход',
 'Лабораторный медицинский анализ (Юниоры)',
 'Лабораторный медицинский анализ',
 'Кузовной ремонт (Юниоры)',
 'Кузовной ремонт',
 'Архитектура',
 'Ландшафтный дизайн (Юниоры)',
 'Ландшафтный дизайн',
 'Кирпичная кладка',
 'Монтаж и эксплуатация газового оборудования',
 'Облицовка плиткой (юниоры)',
 'Облицовка плиткой (16-22)',
 'Бетонные строительные работы. ',
 'Архитектурная обработка камня']

In [31]:
sett=df[df['cluster_kmeans'] == 6]
sett.drop(sett.iloc[:,1:],inplace=True,axis=1)
comp=pd.unique(sett["название_компетенции_wsss"]).tolist()
comp

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


['Предпринимательство (Юниоры)',
 'Предпринимательство',
 'Корпоративная защита от внутренних угроз информационной безопасности',
 'Информационные кабельные сети',
 'Документационное обеспечение управления и архивоведение',
 'Промышленная автоматика',
 'Промышленная робототехника',
 'Сити-фермерство',
 'Технологии информационного моделирования BIM',
 'Электроника (Юниоры)',
 'Электроника ',
 'Интернет вещей',
 'Инженерный дизайн CAD ',
 'Инженерия космических систем',
 'Изготовление прототипов ',
 'Изготовление прототипов (12-14)',
 'Разработка компьютерных игр и мультимедийных приложений ',
 'Разработка виртуальной и дополненной реальности (Юниоры)',
 'Разработка виртуальной и дополненной реальности ',
 'Программные решения для бизнеса',
 'Программные решения для бизнеса(юниоры)',
 'Разработка решений на базе блокчейн технологи',
 'Разработка мобильных приложений',
 'Машинное обучение и большие данные (Юниоры)',
 'Машинное обучение и большие данные ',
 'Информационные кабельные сети(ю

Исходя из этих данных, довольно просто определить каждый кластер по вхождению компетенций. Поскольку это unsupervised learning(без меток), то точность кластеров может разница

In [32]:
#Naming clusters
d = {0:"Творчество и дизайн", 
     1:"Сфера услуг",
     2:"Производство и инженерные технологии", 
     3:"Информационные и коммуникационные технологии", 
     4:"Транспорт и логистика",
     5:"Образование",
     6:"Строительство и строительные технологии"}

# атрибут "named_clusters" содержит в себе данные наименнованых кластеров 
df["named_clusters"] = df["cluster_kmeans"].replace(d)

In [33]:
### Сохранение результируещего набора данных
df.to_csv('dataset_clusterd.csv', index=False, encoding="utf-16", sep="\t")
df.head()

,название_компетенции_wsss,описание_компетенции_wsss,название_раздела_wsss,значимость_раздела_wsss,признаки_раздела_wsss,count_description,count_title,count_titlefeat,cluster_kmeans,cluster_kmeans_full,cluster_ngram2,named_clusters
0,Ювелирное дело,профессия ювелира прежде связана ручным трудом...,обработка поверхности,15.0,специалист должен знать понимать специализиров...,256,2,48,3,5,3,Информационные и коммуникационные технологии
1,Ювелирное дело,профессия ювелира прежде связана ручным трудом...,изготовление сложных элементов ювелирных издел...,20.0,специалист должен знать понимать способы испол...,256,9,38,3,2,3,Информационные и коммуникационные технологии
2,Ювелирное дело,профессия ювелира прежде связана ручным трудом...,изготовление простых элементов ювелирных изделий,20.0,специалист должен знать понимать способы испол...,256,5,15,3,2,3,Информационные и коммуникационные технологии
3,Ювелирное дело,профессия ювелира прежде связана ручным трудом...,подготовка драгоценных сплавов изготовления эл...,10.0,специалист должен знать понимать свойства спос...,256,7,21,3,3,3,Информационные и коммуникационные технологии
4,Ювелирное дело,профессия ювелира прежде связана ручным трудом...,изготовление сплавов драгоценных металлов,5.0,специалист должен знать понимать состав сплаво...,256,4,56,3,3,3,Информационные и коммуникационные технологии


## Классификация исходных компетенций
### Разбиение на тестовую и обучающую выборки

In [34]:
from sklearn.model_selection import train_test_split
y=df["cluster_kmeans"]
X_train, X_test, y_train, y_test = train_test_split(X_full, y, test_size=0.1, random_state=0)

## Итог
* Кластеризация набора данных - было проведено тестирование, а также обучение модели, используя текстовые и числовые переменные
* Визуализация набора данных - данные были визуализированны на основе двух методов зависимостей
* Классификация исходных компетенций - были рассмотрены необходимые методы классификации